# 🏔️ PeakForm — Weekly Fitness & Nutrition Report

**How to use this notebook each Saturday/Sunday:**

1. Export your **MacroFactor** data → Download XLSX → Drop into your Google Drive folder
2. Export your **Garmin Connect** activities → Download CSV → Drop into the same folder
3. Open this notebook and click **Runtime → Run all** (or press `Ctrl+F9`)
4. Your report appears at the bottom in ~30 seconds

---

**Expected Google Drive folder structure:**
```
My Drive/
└── PeakForm/
    ├── macrofactor_export.xlsx   ← MacroFactor XLSX export
    └── garmin_activities.csv    ← Garmin Connect CSV export
```
The notebook auto-detects the newest `.xlsx` and `.csv` in the folder — no renaming needed.

In [ ]:
# ============================================================
# ✏️  CONFIGURATION — only cell you ever need to edit
# ============================================================

# Folder inside your Google Drive that holds the export files
DRIVE_FOLDER = "PeakForm"   # e.g. "PeakForm" or "Fitness/Exports"

# Week to analyse — set to any date within the Mon–Sun week you want.
# Leave as None to default to the current calendar week.
ANALYSIS_WEEK = None   # e.g. "2026-02-16"

# Save the finished report back to your Drive folder? (True / False)
SAVE_REPORT_TO_DRIVE = True

In [ ]:
# ============================================================
# 1. Mount Google Drive
# ============================================================
from google.colab import drive
drive.mount("/content/drive", force_remount=False)
print("✅ Google Drive mounted")

In [ ]:
# ============================================================
# 2. Pull the latest PeakForm agent code from GitHub
# ============================================================
# If your repo is PRIVATE, add a GitHub token to Colab Secrets:
#   1. Click the 🔑 key icon in the left sidebar
#   2. Add a secret named:  GITHUB_TOKEN
#   3. Value: a GitHub Personal Access Token with 'repo' scope
#      (github.com → Settings → Developer settings → Personal access tokens)
#   4. Toggle "Notebook access" ON for this secret
# If your repo is PUBLIC, no token needed — this cell works as-is.
# ============================================================
import os, subprocess, sys

REPO_DIR  = "/content/PeakForm-C"
REPO_HOST = "github.com/bhackerb/PeakForm-C.git"

# Try to load a GitHub token from Colab Secrets (silent if not set)
token = None
try:
    from google.colab import userdata
    token = userdata.get("GITHUB_TOKEN")
except Exception:
    pass

def _build_url(host, tok):
    return f"https://{tok}@{host}" if tok else f"https://{host}"

if os.path.exists(REPO_DIR):
    # Already cloned — update remote URL in case token changed, then pull
    subprocess.run(
        ["git", "-C", REPO_DIR, "remote", "set-url", "origin", _build_url(REPO_HOST, token)],
        capture_output=True
    )
    result = subprocess.run(
        ["git", "-C", REPO_DIR, "pull", "--ff-only"],
        capture_output=True, text=True
    )
    print("✅ Repo updated:", result.stdout.strip() or result.stderr.strip())
else:
    result = subprocess.run(
        ["git", "clone", _build_url(REPO_HOST, token), REPO_DIR],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise RuntimeError(
            "\n❌ Could not clone the repository."
            "\n   The repo is likely private and needs a GitHub token."
            "\n"
            "\n   Fix — add your token to Colab Secrets:"
            "\n   1. Click the 🔑 key icon in the left sidebar"
            "\n   2. Secret name:  GITHUB_TOKEN"
            "\n   3. Value: a Personal Access Token with 'repo' scope"
            "\n      → github.com → Settings → Developer settings"
            "\n        → Personal access tokens → Fine-grained tokens → Generate new token"
            "\n   4. Toggle 'Notebook access' ON"
            "\n   5. Re-run this cell"
            f"\n\n   Git output: {result.stderr.strip()}"
        )
    print("✅ Repo cloned", "(authenticated)" if token else "(public)")

# Add to Python path
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

In [ ]:
# ============================================================
# 3. Install dependencies (cached after first run)
# ============================================================
!pip install -q openpyxl pandas numpy rich
print("✅ Dependencies ready")

In [ ]:
# ============================================================
# 4. Locate export files in Drive (auto-detect newest)
# ============================================================
import pathlib
from datetime import datetime

drive_root = pathlib.Path("/content/drive/MyDrive")
folder = drive_root / DRIVE_FOLDER

if not folder.exists():
    raise FileNotFoundError(
        f"\n❌ Drive folder not found: {folder}"
        f"\n   Create a folder called '{DRIVE_FOLDER}' in your Google Drive "
        f"and drop your export files there."
    )

def _newest(folder, suffix):
    """Return the most recently modified file with the given suffix."""
    candidates = sorted(
        folder.glob(f"*{suffix}"),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    return candidates[0] if candidates else None

mf_file    = _newest(folder, ".xlsx")
garmin_file = _newest(folder, ".csv")

if not mf_file:
    raise FileNotFoundError(
        f"\n❌ No .xlsx file found in Drive/{DRIVE_FOLDER}/"
        "\n   Export MacroFactor data and drop the XLSX into that folder."
    )
if not garmin_file:
    raise FileNotFoundError(
        f"\n❌ No .csv file found in Drive/{DRIVE_FOLDER}/"
        "\n   Export Garmin Connect activities and drop the CSV into that folder."
    )

mf_mtime    = datetime.fromtimestamp(mf_file.stat().st_mtime).strftime("%Y-%m-%d %H:%M")
gar_mtime   = datetime.fromtimestamp(garmin_file.stat().st_mtime).strftime("%Y-%m-%d %H:%M")

print(f"✅ MacroFactor : {mf_file.name}  (modified {mf_mtime})")
print(f"✅ Garmin      : {garmin_file.name}  (modified {gar_mtime})")

In [ ]:
# ============================================================
# 5. Run the PeakForm agent
# ============================================================
from peakform.agent import run as peakform_run

print("⏳ Running analysis...")
report_md = peakform_run(
    mf_filepath=str(mf_file),
    garmin_filepath=str(garmin_file),
    week=ANALYSIS_WEEK,
    verbose=True,
)
print("✅ Analysis complete")

In [ ]:
# ============================================================
# 6. Display the weekly report
# ============================================================
from IPython.display import Markdown, display
display(Markdown(report_md))

In [ ]:
# ============================================================
# 7. (Optional) Save report back to Google Drive
# ============================================================
if SAVE_REPORT_TO_DRIVE:
    from datetime import date
    today = date.today().isoformat()
    out_path = folder / f"peakform_report_{today}.md"
    out_path.write_text(report_md, encoding="utf-8")
    print(f"✅ Report saved to Drive/{DRIVE_FOLDER}/{out_path.name}")
else:
    print("ℹ️  Report not saved (SAVE_REPORT_TO_DRIVE = False)")